In [ ]:
!pip install pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=ec892b4094e845c4a869c40e8fa48cc00333c1dca11b28f8a6d73599e8d3dc3b
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy.stats import tukey_hsd
import scipy.stats as stats
from scipy.stats import spearmanr
import statsmodels.api as sm
from statsmodels.formula.api import ols



## Pull in TMS data sheet consisting of trials

This data sheet is organized to contain response times and accuracy score for every problem viewed by any given participant. If a participant did not see a problem, it was not included in the data sheet. Each row represents a problem viewed and solved by a participant. Each row therefore contains participant ID, treatment types (coded as A,B,C where A is for M1 region treatment, B is SMA treatment, and C is our active control), stimuli version (1,2,3) problem number order, problem (name of stimuli), problem type (array, mental, coding, trees), response time, accuracy, session number (1,2,3), percieved mental rotation difficult of the current session, percieved program task difficulty of the current session, percived changes in difficulty for mental rotations of the current session, percived changes in difficulty for programming tasks of the current session, age, gender, occupation, years of programming experiance, and number of programming languages known.

In [ ]:

df = pd.read_csv("TMSdata.csv")

df

,ID,treatment,version,problemNumber,problem,problemType,responseTime,accuracy,sessionNo,mrDifficulty,changeMrDifficulty,programDifficulty,changeProgramDifficulty,age,gender,race,occupation,yearsProgramming,numPL
0,p32420,A,version1,48,205.bmp,ARRAY,9.205110,1,NaN,NaN,NaN,NaN,NaN,27.0,Female,Asian,Part-time or full-tume job that does not invol...,3.0,2.0
1,p32420,A,version1,50,6.BMP,ARRAY,5.005046,0,NaN,NaN,NaN,NaN,NaN,27.0,Female,Asian,Part-time or full-tume job that does not invol...,3.0,2.0
2,p32420,A,version1,18,24.BMP,ARRAY,2.689130,0,NaN,NaN,NaN,NaN,NaN,27.0,Female,Asian,Part-time or full-tume job that does not invol...,3.0,2.0
3,p32420,A,version1,38,10.BMP,ARRAY,4.440007,0,NaN,NaN,NaN,NaN,NaN,27.0,Female,Asian,Part-time or full-tume job that does not invol...,3.0,2.0
4,p32420,A,version1,33,201.bmp,ARRAY,11.836844,1,NaN,NaN,NaN,NaN,NaN,27.0,Female,Asian,Part-time or full-tume job that does not invol...,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,p04012,A,version2,31,320.bmp,TREE,57.775143,0,1.0,1.0,NaN,3.0,NaN,23.0,Male,White,Full-time job that involves programming,6.0,1.0
2747,p04012,A,version2,47,323.bmp,TREE,35.931511,1,1.0,1.0,NaN,3.0,NaN,23.0,Male,White,Full-time job that involves programming,6.0,1.0
2748,p04012,A,version2,48,325.bmp,CODING,111.818344,1,1.0,1.0,NaN,3.0,NaN,23.0,Male,White,Full-time job that involves programming,6.0,1.0
2749,p04012,A,version2,5,329.bmp,ARRAY,44.184052,1,1.0,1.0,NaN,3.0,NaN,23.0,Male,White,Full-time job that involves programming,6.0,1.0


---

We removed some trials for participant p19391 as their response times were 2 standard deviations away from the mean.


In [ ]:
#REMOVES OUTLIERS OF PARTICIAPNT 19391
sleepyP = df.query("ID == 'p19391'")
for index in sleepyP.index:
  data_point = sleepyP["responseTime"][index]
  problem = sleepyP["problem"][index]
  mean = df.query("problem==@problem")["responseTime"].mean()
  std_dev = df.query("problem==@problem")["responseTime"].std()
  z_score = (data_point - mean) / std_dev
  if abs(z_score) > 2:
      mask = (df['ID'] == 'p19391') & (df['problem'] == problem)
      df = df[~mask]
  else:
      pass





---

The following contains demographic information pulled from the survey data

In [ ]:
print("DataFrame length: ", len(df))
print("Num Participants: ", len(df["ID"].unique()))

count = df[['ID','gender']].drop_duplicates()
count2 = df[['ID','occupation']].drop_duplicates()
count3 = df[['ID','race']].drop_duplicates()
print("male: ", count['gender'].value_counts()["Male"])
print("female: ", count['gender'].value_counts()["Female"])
print()
print("undergrad: ", count2['occupation'].value_counts()["Computing Student (undergraduate)"])
print("grad: ", count2['occupation'].value_counts()["Computing Student (graduate)"])
print("Full-TIme: ", count2['occupation'].value_counts()["Full-time job that involves programming"])
print("Non-comptuing: ", count2['occupation'].value_counts()['Non-computing student'])
print("Non-comptuing Full-Time: ", count2['occupation'].value_counts()["Part-time or full-tume job that does not involve programming"])
print()
print("white: ", count3['race'].value_counts()["White"])
print("asian: ", count3['race'].value_counts()["Asian"])




DataFrame length:  2726
Num Participants:  16
male:  8
female:  7

undergrad:  6
grad:  4
Full-TIme:  2
Non-comptuing:  1
Non-comptuing Full-Time:  1

white:  5
asian:  10


# Replication and Coherency:

- Have 14 full participants and 2 participant with two treatments (i.e., 19391, 37340)
List = [00001, 00047, 00302, 00071, 00193, 00209, 03029, 00026, 00782, 00481, 04012, 13012, 19391, 18602, 32420, 37340]

(a) Computed a one-way ANOVA (f_oneway) of “responses times for Mental Rotation questions that participants got right” across three treatment groups. What are the p and f values?



In [ ]:
A = df.query("accuracy==1 & treatment=='A' & problemType=='MENTAL'")["responseTime"]
B = df.query("accuracy==1 & treatment=='B' & problemType=='MENTAL'")["responseTime"]
C = df.query("accuracy==1 & treatment=='C' & problemType=='MENTAL'")["responseTime"]
#res = tukey_hsd(A, B, C ) -> post-hoc tests if we want to look further into groups

fvalue, pvalue = stats.f_oneway(A, B, C)
print("fvalue: ",fvalue, "pvalue: ",pvalue)

len(df.query("accuracy==1 & problemType=='MENTAL'"))





fvalue:  0.746409667987899 pvalue:  0.47440472060705674


781

# Analysis - Primary RQs

(1) Does there exist a question type T such that “raw accuracies on T for one of the treatment conditions are statistically different from raw accuracies on T for another treatment condition (considering all participants)”?

What are the statistical significance levels and effect sizes?

**RESULT** -> No, there does not exist a question type T such that raw accuracies on T for one treatment condition are statiscally different from raw accuracies on T for another treatment condition when looking at all participants


In [ ]:
import numpy as np
from scipy.stats import chi2_contingency

In [ ]:
#Cramer's V is a measure of association for nominal (categorical) variables and
#provides an indication of the strength of the relationship between two categorical variables.
#It ranges from 0 to 1, where 0 means no association, and 1 indicates a perfect association.

def cramers_v(crosstab):
    chi2 = chi2_contingency(crosstab)[0]
    n = crosstab.sum().sum()
    min_dim = min(crosstab.shape) - 1
    return np.sqrt(chi2 / (n * min_dim))

In [ ]:
# Chi squared test needed to compare two categorical varaibles (accuracy and treatment) 3x3

crosstab = pd.crosstab(index = [df['accuracy'], df['problemType']], columns = df['treatment'])
effect_size = cramers_v(crosstab)
print("RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy\n")
print(crosstab, "\n")

print("Cramer's V Effect Size:", effect_size, "\n")
stats.chi2_contingency(crosstab)


RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy

treatment               A    B    C
accuracy problemType               
0        ARRAY         19   16   19
         CODING        19   22   21
         MENTAL        17   21   23
         TREE          30   38   41
1        ARRAY        151  156  166
         CODING       162  164  176
         MENTAL       258  259  264
         TREE         224  228  232 

Cramer's V Effect Size: 0.022553335243283044 



Chi2ContingencyResult(statistic=2.7731757776089174, pvalue=0.9994115186078658, dof=14, expected_freq=array([[ 17.432135  ,  17.90755686,  18.66030814],
       [ 20.01467351,  20.56052825,  21.42479824],
       [ 19.6918562 ,  20.22890682,  21.07923698],
       [ 35.18708731,  36.14673514,  37.66617755],
       [152.69258988, 156.85693324, 163.45047689],
       [162.054292  , 166.47395451, 173.47175348],
       [252.12032282, 258.99633162, 269.88334556],
       [220.80704329, 226.82905356, 236.36390315]]))

In [ ]:
# Chi square (2x2) one problem at a time - mental rotation tasks

mental = df.query("problemType=='MENTAL' ")

crosstab = pd.crosstab(index = [mental['accuracy']], columns = mental['treatment'])
effect_size = cramers_v(crosstab)
print("RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy\n")
print(crosstab, "\n")

print("Cramer's V Effect Size:", effect_size, "\n")
stats.chi2_contingency(crosstab)


RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy

treatment    A    B    C
accuracy                
0           17   21   23
1          258  259  264 

Cramer's V Effect Size: 0.029689714703444423 



Chi2ContingencyResult(statistic=0.74220545202276, pvalue=0.6899730597377615, dof=2, expected_freq=array([[ 19.92280285,  20.28503563,  20.79216152],
       [255.07719715, 259.71496437, 266.20783848]]))

In [ ]:
# Chi square (2x2) one problem at a time - coding tasks

coding = df.query("problemType=='CODING' ")

crosstab = pd.crosstab(index = [coding['accuracy']], columns = coding['treatment'])
effect_size = cramers_v(crosstab)
print("RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy\n")
print(crosstab, "\n")

print("Cramer's V Effect Size:", effect_size, "\n")
stats.chi2_contingency(crosstab)


RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy

treatment    A    B    C
accuracy                
0           19   22   21
1          162  164  176 

Cramer's V Effect Size: 0.018846760578347197 



Chi2ContingencyResult(statistic=0.20033301674381368, pvalue=0.9046867675732717, dof=2, expected_freq=array([[ 19.89716312,  20.44680851,  21.65602837],
       [161.10283688, 165.55319149, 175.34397163]]))

In [ ]:
# Chi square (2x2) one problem at a time - data structure manipulation

tree = df.query("(problemType=='TREE' | problemType=='ARRAY')")

crosstab = pd.crosstab(index = [tree['accuracy']], columns = tree['treatment'])
effect_size = cramers_v(crosstab)
print("RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy\n")
print(crosstab, "\n")

print("Cramer's V Effect Size:", effect_size, "\n")
stats.chi2_contingency(crosstab)


RESULT: there are no statistical significant relationships between treatment, problem type, and accuracy

treatment    A    B    C
accuracy                
0           49   54   60
1          375  384  398 

Cramer's V Effect Size: 0.019169623625703482 



Chi2ContingencyResult(statistic=0.4850663003354904, pvalue=0.7846377362750085, dof=2, expected_freq=array([[ 52.35757576,  54.08636364,  56.55606061],
       [371.64242424, 383.91363636, 401.44393939]]))



---



(2) Does there exist a question type T such that “raw response times on T for one of the treatment conditions are statistically different from raw response times on T for another treatment condition (considering all participants)”?

**Result** -> No, there does not exist a question type T that meets such requirements. Note, I looked at response times for correct answers




In [ ]:
#One Way Anova - we have one independent variable (treatments) with continous data

A = df.query("accuracy==1.0 & treatment=='A' & problemType=='MENTAL'")["responseTime"]
B = df.query("accuracy==1.0 & treatment=='B' & problemType=='MENTAL'")["responseTime"]
C = df.query("accuracy==1.0 & treatment=='C' & problemType=='MENTAL'")["responseTime"]
fvalue, pvalue = stats.f_oneway(A,C )
print("MENTAL ROTATION Tasks (Treatment A vs Control C) -> fvalue: ",fvalue, "pvalue: ",pvalue)
fvalue, pvalue = stats.f_oneway(B,C )
print("MENTAL ROTATION Tasks (Treatment B vs Control C) -> fvalue: ",fvalue, "pvalue: ",pvalue)

MENTAL ROTATION Tasks (Treatment A vs Control C) -> fvalue:  0.1232702553663312 pvalue:  0.7256565502293004
MENTAL ROTATION Tasks (Treatment B vs Control C) -> fvalue:  1.5055792371350047 pvalue:  0.22036907040310497


In [ ]:
#One Way Anova - we have one independent variable (treatments) with continous data
#Now: Considering response times of correct answers only

A = df.query("accuracy==1 & treatment=='A' & problemType=='CODING'")["responseTime"]
B = df.query("accuracy==1 & treatment=='B' & problemType=='CODING'")["responseTime"]
C = df.query("accuracy==1 & treatment=='C' & problemType=='CODING'")["responseTime"]
fvalue, pvalue = stats.f_oneway(A,C )
print("CODING Tasks (Treatment A vs Control C) -> fvalue: ",fvalue, "pvalue: ",pvalue)
fvalue, pvalue = stats.f_oneway(B,C )
print("CODING Tasks (Treatment B vs Control C) -> fvalue: ",fvalue, "pvalue: ",pvalue)



CODING Tasks (Treatment A vs Control C) -> fvalue:  0.06305094546570085 pvalue:  0.8018906929068079
CODING Tasks (Treatment B vs Control C) -> fvalue:  0.005547182751844167 pvalue:  0.9406729819305629


In [ ]:
A = df.query("accuracy==1.0 & treatment=='A' & (problemType=='TREE' | problemType=='ARRAY')")["responseTime"]
B = df.query("accuracy==1.0 & treatment=='B' & (problemType=='TREE' | problemType=='ARRAY')")["responseTime"]
C = df.query("accuracy==1.0 & treatment=='C' & (problemType=='TREE' | problemType=='ARRAY')")["responseTime"]
fvalue, pvalue = stats.f_oneway(A,C )
print("DATA STRUCTURE Tasks (Treatment A vs Control C) -> fvalue: ",fvalue, "pvalue: ",pvalue)
fvalue, pvalue = stats.f_oneway(B,C )
print("DATA STRUCTURE Tasks (Treatment B vs Control C) -> fvalue: ",fvalue, "pvalue: ",pvalue)

DATA STRUCTURE Tasks (Treatment A vs Control C) -> fvalue:  0.09226687374669114 pvalue:  0.7613967895517975
DATA STRUCTURE Tasks (Treatment B vs Control C) -> fvalue:  0.21056485297952213 pvalue:  0.6464528758032904




---

(3) Let AbsDelta(x) be the absolute value of the difference between “the average of x in the control condition and this one observation of x in a treatment condition”.

Does there exist a question type T such that “AbsDelta(response time) on T for treatment condition A is statistically different from AbsDelta(response time) on T for treatment condition B”?

Alternate phrasing: make an additional column in your dataframe for AbsDelta(response time) and use it for statistics.



*NOTE: I am going to compute the average time of correct responses as my base. I will be computing the deltas for times that are of correct responses*

**RESULT** -> AbsDelta of question type 'CODING' for treatment A is statisitcally different from AbsDelta of question type 'CODING' on treatment B (p < 0.05).


In [ ]:
# Make new dataframe with average times per problem type T across treatments
# NOTE! I am looking at RESPONSE TIMES FOR CORRECT ANSWERS

pID = df['ID'].unique()
treatments = df['treatment'].unique()
problemTypes = df['problemType'].unique()
averageTimes = []
for id in pID:
    for treat in treatments:
        for problem in problemTypes:
            average = df.query("ID==@id & treatment==@treat & problemType==@problem & accuracy==1")["responseTime"].mean()
            averageTimes.append([id, treat, problem ,average])

column_names=["ID","treatment",'problemType', 'avgTime']
df_avg=pd.DataFrame(averageTimes, columns=column_names)
df_avg = df_avg.set_index(['ID', 'treatment', 'problemType'])

df_avg


avgTime
ID     treatment problemType           
p32420 A         ARRAY        17.892599
                 TREE         22.077676
                 MENTAL        8.897606
                 CODING       20.403861
       B         ARRAY         8.021723
...                                 ...
p04012 B         CODING       26.915436
       C         ARRAY        11.068279
                 TREE         18.574696
                 MENTAL        8.594950
                 CODING       19.795748

[192 rows x 1 columns]

In [ ]:
def getDeltasAbs():
  deltas = []
  for index in df.index:
    id = df['ID'][index]
    treat = df['treatment'][index]
    problemType = df['problemType'][index]
    if treat != 'C':
      base = df_avg.loc[id, 'C', problemType]
      T_i = df['responseTime'][index]
      delta = abs(base[0] - T_i)
      deltas.append(delta)
    else:
      delta = 0
      deltas.append(delta)
  df['absDelta'] = deltas


getDeltasAbs()




In [ ]:
def getDeltas():
  deltas = []
  for index in df.index:
    id = df['ID'][index]
    treat = df['treatment'][index]
    problemType = df['problemType'][index]
    if treat != 'C':
      base = df_avg.loc[id, 'C', problemType]
      T_i = df['responseTime'][index]
      delta = T_i - base[0]
      deltas.append(delta)
    else:
      delta = 0
      deltas.append(delta)
  df['delta'] = deltas


getDeltas()

#Positive delta == impaired, negative delta == improved

In [ ]:
A = df.query("treatment=='A' & problemType=='MENTAL' & accuracy==1")["absDelta"]
B = df.query("treatment=='B' & problemType=='MENTAL' & accuracy==1")["absDelta"]
fvalue, pvalue = stats.f_oneway(A, B)

print("Mental Rotation response time absolute deltas: ", fvalue, pvalue)




Mental:  0.03632184048609958 0.8489274938321582


In [ ]:

print("Coding Task: ")
correct = df.query("(treatment=='A' | treatment=='B') & problemType=='CODING' & accuracy==1")
aov = pg.anova(dv='absDelta', between='treatment', data=correct, detailed=True)
aov




Coding Task: 


,Source,SS,DF,MS,F,p-unc,np2
0,treatment,685.012552,1,685.012552,4.821094,0.028822,0.014662
1,Within,46036.036350,324,142.086532,NaN,NaN,NaN


In [ ]:
A = df.query("treatment=='A' & (problemType=='TREE' | problemType=='ARRAY') & accuracy==1")["absDelta"]
B = df.query("treatment=='B' & (problemType=='TREE' | problemType=='ARRAY') & accuracy==1")["absDelta"]
fvalue, pvalue = stats.f_oneway(A, B)

print("Data Structures: ", fvalue, pvalue)


Data Structures:  0.006041445666568266 0.9380659646179409




---

# Analysis - Context

(a) Choose question type T. Among participant who were impaired by TMS on question type T (e.g., for response time), what are the mean, median and maximum impairments, normalized to the unimpaired mean?

Note: looking at response times for correct anwsers only

I define impaired as particpants whose average response time on treatments are longer than that of control




In [ ]:
def percentChange(new, old):
  return(((new-old)/old)*100)

In [ ]:
def getImpaired(problem, treatment):
  pID = df['ID'].unique()
  impaired_pID_treatment = []
  control_ = []

  impaired_both = []
  control_both = []

  if treatment != 'both':
    for id in pID:
      control_avg = df_avg.loc[id, 'C', problem][0]
      treatment_avg = df_avg.loc[id, treatment, problem][0]
      if control_avg < treatment_avg :
        impaired_pID_treatment.append(id)
        control_.append(id)
    return impaired_pID_treatment, control_

  elif treatment == 'either':
    for id in pID:
      control_avg = df_avg.loc[id, 'C', problem][0]
      A_avg = df_avg.loc[id, 'A', problem][0]
      B_avg = df_avg.loc[id, 'B', problem][0]
      if control_avg < A_avg or control_avg < B_avg:
        impaired_both.append(id)
        control_both.append(id)
    return impaired_both, control_both

  else:
    for id in pID:
      control_avg = df_avg.loc[id, 'C', problem][0]
      A_avg = df_avg.loc[id, 'A', problem][0]
      B_avg = df_avg.loc[id, 'B', problem][0]
      if control_avg < A_avg and control_avg < B_avg:
        impaired_both.append(id)
        control_both.append(id)
    return impaired_both, control_both


In [ ]:
def printResults(problem, treatment):
  if treatment != 'both':
    impaired_A, control = getImpaired(problem, treatment)
    impaired_A_avg = df[df['ID'].isin(impaired_A) & (df['treatment'] == treatment) & (df['problemType'] == problem) & (df['accuracy'] == 1)]["responseTime"].mean()
    impaired_A_max = df[df['ID'].isin(impaired_A) & (df['treatment'] == treatment)& (df['problemType'] == problem) & (df['accuracy'] == 1)]["responseTime"].max()
    impaired_control = df[df['ID'].isin(control) & (df['treatment'] == 'C') & (df['problemType'] == problem) &(df['accuracy'] == 1)]["responseTime"].mean()
    print("Num particiapnts: ",len(impaired_A))
    print("In treatment ", treatment, ":")
    print("Control Avg: ", impaired_control," Treatment Avg: ", impaired_A_avg,"(" + str(percentChange(impaired_A_avg, impaired_control)) + "%)" )
    current_large = 0

  else:
    impaired_A, control = getImpaired(problem, 'both')
    impaired_A_avg = df[df['ID'].isin(impaired_A) & (df['treatment'].isin(['A', 'B'])) & (df['problemType'] == problem) & (df['accuracy'] == 1)]["responseTime"].mean()
    impaired_A_max = df[df['ID'].isin(impaired_A) & (df['treatment'].isin(['A', 'B'])) & (df['problemType'] == problem) & (df['accuracy'] == 1)]["responseTime"].max()
    impaired_control = df[df['ID'].isin(control) & (df['treatment'] == 'C') & (df['problemType'] == problem) &(df['accuracy'] == 1)]["responseTime"].mean()
    print("Num particiapnts: ",len(impaired_A))
    print("In Both Treatments:")
    print("Control Avg: ", impaired_control," Treatment Avg: ", impaired_A_avg,"(" + str(percentChange(impaired_A_avg, impaired_control)) + "%)" )




In [ ]:
problem = 'MENTAL'
print("Num participants that were impaired by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')


Num participants that were impaired by each treatment or both for problem type:  MENTAL
Num particiapnts:  9
In treatment  A :
Control Avg:  11.720454321236446  Treatment Avg:  15.076616740994915 (28.635088092766114%)

Num particiapnts:  10
In treatment  B :
Control Avg:  11.925567635367898  Treatment Avg:  15.71820012870289 (31.802532250851563%)

Num particiapnts:  6
In Both Treatments:
Control Avg:  10.868252400124428  Treatment Avg:  14.81054130665542 (36.27343901661557%)


In [ ]:
problem = 'CODING'

print("Num participants that were impaired by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')


Num participants that were impaired by each treatment or both for problem type:  CODING
Num particiapnts:  7
In treatment  A :
Control Avg:  21.484772036715253  Treatment Avg:  28.412041775974227 (32.24269602405363%)

Num particiapnts:  10
In treatment  B :
Control Avg:  22.780796337971644  Treatment Avg:  26.11565030742259 (14.638882328675765%)

Num particiapnts:  7
In Both Treatments:
Control Avg:  21.484772036715253  Treatment Avg:  27.186425580918414 (26.538115156444874%)


In [ ]:
problem = 'TREE'

print("Num participants that were impaired by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')


Num participants that were impaired by each treatment or both for problem type:  TREE
Num particiapnts:  7
In treatment  A :
Control Avg:  17.766014976704373  Treatment Avg:  24.139142403168428 (35.87257713573245%)

Num particiapnts:  9
In treatment  B :
Control Avg:  20.22796693328739  Treatment Avg:  26.48043846539969 (30.910034373366287%)

Num particiapnts:  5
In Both Treatments:
Control Avg:  17.384727824818004  Treatment Avg:  23.99214399614798 (38.00701534077165%)


In [ ]:
problem = 'ARRAY'

print("Num participants that were impaired by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')

Num participants that were impaired by each treatment or both for problem type:  ARRAY
Num particiapnts:  5
In treatment  A :
Control Avg:  15.603008711779559  Treatment Avg:  21.355588049258827 (36.86839790800305%)

Num particiapnts:  5
In treatment  B :
Control Avg:  11.430545505724455  Treatment Avg:  16.51001125803589 (44.437649539714656%)

Num particiapnts:  2
In Both Treatments:
Control Avg:  10.57371125013932  Treatment Avg:  20.779299573464826 (96.51850785305902%)




---

Repeat this analysis for participants who were improved by TMS.


In [ ]:
def getImproved(problem, treatment):
  pID = df['ID'].unique()
  improved_pID_treatment = []
  control_ = []

  improved_both = []
  control_both = []

  if treatment != 'both':
    for id in pID:
      control_avg = df_avg.loc[id, 'C', problem][0]
      treatment_avg = df_avg.loc[id, treatment, problem][0]
      if control_avg > treatment_avg :
        improved_pID_treatment.append(id)
        control_.append(id)
    return improved_pID_treatment, control_

  else:
    for id in pID:
      control_avg = df_avg.loc[id, 'C', problem][0]
      A_avg = df_avg.loc[id, 'A', problem][0]
      B_avg = df_avg.loc[id, 'B', problem][0]
      if control_avg > A_avg and control_avg > B_avg:
        improved_both.append(id)
        control_both.append(id)
    return improved_both, control_both


In [ ]:
def printResults_improve(problem, treatment):
  if treatment != 'both':
    improved_A, control = getImproved(problem, treatment)
    improved_A_avg = df[df['ID'].isin(improved_A) & (df['treatment'] == treatment) & (df['problemType'] == problem) & (df['accuracy'] == 1)]["responseTime"].mean()
    improved_control = df[df['ID'].isin(control) & (df['treatment'] == 'C') & (df['problemType'] == problem) &(df['accuracy'] == 1)]["responseTime"].mean()
    print("Num particiapnts: ",len(improved_A))
    print("In treatment ", treatment, ":")
    print("Control Avg: ", improved_control," Treatment Avg: ", improved_A_avg,"(" + str(percentChange(improved_A_avg, improved_control)) + "%)" )
    #print(df[df['responseTime'] == impaired_A_max]['ID'])
    current_large = 0

  else:
    improved_A, control = getImproved(problem, 'both')
    improved_A_avg = df[df['ID'].isin(improved_A) & (df['treatment'].isin(['A', 'B'])) & (df['problemType'] == problem) & (df['accuracy'] == 1)]["responseTime"].mean()
    improved_control = df[df['ID'].isin(control) & (df['treatment'] == 'C') & (df['problemType'] == problem) &(df['accuracy'] == 1)]["responseTime"].mean()

    print("Num particiapnts: ",len(improved_A))
    print("In Both Treatments:")
    print("Control Avg: ", improved_control," Treatment Avg: ", improved_A_avg,"(" + str(percentChange(improved_A_avg, improved_control)) + "%)" )







In [ ]:
problem = 'MENTAL'

print("Num participants that were improved by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')

Num participants that were improved by each treatment or both for problem type:  MENTAL
Num particiapnts:  9
In treatment  A :
Control Avg:  11.720454321236446  Treatment Avg:  15.076616740994915 (28.635088092766114%)

Num particiapnts:  10
In treatment  B :
Control Avg:  11.925567635367898  Treatment Avg:  15.71820012870289 (31.802532250851563%)

Num particiapnts:  6
In Both Treatments:
Control Avg:  10.868252400124428  Treatment Avg:  14.81054130665542 (36.27343901661557%)


In [ ]:
problem = 'CODING'

print("Num participants that were improved by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')

Num participants that were improved by each treatment or both for problem type:  CODING
Num particiapnts:  7
In treatment  A :
Control Avg:  21.484772036715253  Treatment Avg:  28.412041775974227 (32.24269602405363%)

Num particiapnts:  10
In treatment  B :
Control Avg:  22.780796337971644  Treatment Avg:  26.11565030742259 (14.638882328675765%)

Num particiapnts:  7
In Both Treatments:
Control Avg:  21.484772036715253  Treatment Avg:  27.186425580918414 (26.538115156444874%)


In [ ]:
problem = 'TREE'

print("Num participants that were improved by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')

Num participants that were improved by each treatment or both for problem type:  TREE
Num particiapnts:  7
In treatment  A :
Control Avg:  17.766014976704373  Treatment Avg:  24.139142403168428 (35.87257713573245%)

Num particiapnts:  9
In treatment  B :
Control Avg:  20.22796693328739  Treatment Avg:  26.48043846539969 (30.910034373366287%)

Num particiapnts:  5
In Both Treatments:
Control Avg:  17.384727824818004  Treatment Avg:  23.99214399614798 (38.00701534077165%)


In [ ]:
problem = 'ARRAY'

print("Num participants that were improved by each treatment or both for problem type: ", problem)
printResults(problem, 'A' )
print()
printResults(problem, 'B')
print()
printResults(problem, 'both')

Num participants that were improved by each treatment or both for problem type:  ARRAY
Num particiapnts:  5
In treatment  A :
Control Avg:  15.603008711779559  Treatment Avg:  21.355588049258827 (36.86839790800305%)

Num particiapnts:  5
In treatment  B :
Control Avg:  11.430545505724455  Treatment Avg:  16.51001125803589 (44.437649539714656%)

Num particiapnts:  2
In Both Treatments:
Control Avg:  10.57371125013932  Treatment Avg:  20.779299573464826 (96.51850785305902%)




---





In [ ]:
#Differences in absolute deltas of response times over treatment and problem types DATA STRUCTURE
data1 = df.query("treatment=='A' & accuracy==1 & (problemType =='TREE' | problemType =='ARRAY') & occupation=='Computing Student (undergraduate)'")["absDelta"]
data2 = df.query("treatment=='A' & accuracy==1 & (problemType =='TREE' | problemType =='ARRAY') & occupation=='Computing Student (graduate)'")["absDelta"]
fvalue, pvalue = stats.f_oneway(data1, data2)
print("fvalue: ",fvalue, "pvalue: ",pvalue)
print("Average:")
print("Grad CS (MEAN): ", data2.mean())
print("undergrad CS (MEAN): ", data1.mean())



fvalue:  3.1473877718095067 pvalue:  0.07723257905062081
Average:
Grad CS (MEAN):  8.221003188278484
undergrad CS (MEAN):  10.304022067827194




---


Look if second half of responses (over all people) are faster than first half in s.s. Manner?


In [ ]:
firstHalf = df.query("treatment == 'A' & problemNumber < 30 & problemType=='TREE' ")['responseTime']
secondHalf = df.query("treatment == 'A' & problemNumber > 30 & problemType=='TREE' ")['responseTime']

fvalue, pvalue = stats.f_oneway(firstHalf, secondHalf)
print( fvalue, pvalue)
print(firstHalf.mean(), secondHalf.mean())


#from sklearn.preprocessing import LabelEncoder

impaired = df.query("treatment=='A' & accuracy==1 & problemType =='TREE'")
impaired = impaired[impaired['sessionNo'].notna() ]

ordinal_mapping = {'version1': 1, 'version2': 2, 'version3': 3}
impaired['version'] = impaired['version'].map(ordinal_mapping)


X = impaired[['version', 'sessionNo', 'problemNumber']]
y = impaired['responseTime']

model = sm.OLS(y, X).fit()
print(model.summary())



4.257840120057699 0.040114553350282404
22.593954414768405 27.439966249850148
                                 OLS Regression Results                                
Dep. Variable:           responseTime   R-squared (uncentered):                   0.629
Model:                            OLS   Adj. R-squared (uncentered):              0.623
Method:                 Least Squares   F-statistic:                              107.5
Date:                Wed, 26 Jul 2023   Prob (F-statistic):                    1.02e-40
Time:                        19:52:14   Log-Likelihood:                         -815.96
No. Observations:                 193   AIC:                                      1638.
Df Residuals:                     190   BIC:                                      1648.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err        



---



#TMS and participant perception questions

(1) Participants were asked to rate the diffciculty of the problems for the current session. They were also asked to rate the difficulty in comparison to the previous session. These are represented in the form of a survey. Percieved difficulty changes were evaluated via statements such as "less", "more", and "same". Difficulty of the current session were rated in terms of a 5-scale Likert.

In [ ]:
from sklearn.preprocessing import LabelEncoder

sub = df[['ID','changeMrDifficulty', 'sessionNo', 'treatment']].drop_duplicates()
sub = sub.query('sessionNo != 1')

crosstab = pd.crosstab(index = [sub['changeMrDifficulty'], sub['sessionNo']], columns = sub['treatment'])
print(crosstab, "\n")

stats.chi2_contingency(crosstab)



treatment                     A  B  C
changeMrDifficulty sessionNo         
less               2.0        3  1  5
                   3.0        3  2  1
more               2.0        1  0  0
                   3.0        2  2  0
same               2.0        0  3  1
                   3.0        1  1  1 



Chi2ContingencyResult(statistic=11.039583333333335, pvalue=0.35444242317929997, dof=10, expected_freq=array([[3.33333333, 3.        , 2.66666667],
       [2.22222222, 2.        , 1.77777778],
       [0.37037037, 0.33333333, 0.2962963 ],
       [1.48148148, 1.33333333, 1.18518519],
       [1.48148148, 1.33333333, 1.18518519],
       [1.11111111, 1.        , 0.88888889]]))

In [ ]:
sub = df[['ID','changeProgramDifficulty', 'sessionNo', 'treatment']].drop_duplicates()
sub = sub.query('sessionNo != 1')

crosstab = pd.crosstab(index = [sub['changeProgramDifficulty']], columns = sub['sessionNo'])

print(crosstab, "\n")

stats.chi2_contingency(crosstab)



sessionNo                2.0  3.0
changeProgramDifficulty          
less                       9    6
more                       1    4
same                       4    3 



Chi2ContingencyResult(statistic=2.509262166405024, pvalue=0.28518103689516255, dof=2, expected_freq=array([[7.77777778, 7.22222222],
       [2.59259259, 2.40740741],
       [3.62962963, 3.37037037]]))

In [ ]:
sub = df[['ID','mrDifficulty', 'sessionNo', 'treatment']].drop_duplicates()

crosstab = pd.crosstab(index = [sub['mrDifficulty']], columns = sub['sessionNo'])

print(crosstab, "\n")


stats.chi2_contingency(crosstab)



sessionNo     1.0  2.0  3.0
mrDifficulty               
1.0             1    3    3
2.0             4    4    3
3.0             0    4    3
4.0             6    3    4
5.0             1    0    0 



Chi2ContingencyResult(statistic=8.232196375053519, pvalue=0.4111236008115166, dof=8, expected_freq=array([[2.15384615, 2.51282051, 2.33333333],
       [3.38461538, 3.94871795, 3.66666667],
       [2.15384615, 2.51282051, 2.33333333],
       [4.        , 4.66666667, 4.33333333],
       [0.30769231, 0.35897436, 0.33333333]]))

In [ ]:
sub = df[['ID','programDifficulty', 'sessionNo', 'treatment']].drop_duplicates()

crosstab = pd.crosstab(index = [sub['programDifficulty']], columns = sub['sessionNo'])

print(crosstab, "\n")


stats.chi2_contingency(crosstab)


sessionNo          1.0  2.0  3.0
programDifficulty               
1.0                  2    1    2
2.0                  3    3    5
3.0                  4    7    5
4.0                  3    2    1
5.0                  0    1    0 



Chi2ContingencyResult(statistic=4.800243506493506, pvalue=0.7786974624959159, dof=8, expected_freq=array([[1.53846154, 1.79487179, 1.66666667],
       [3.38461538, 3.94871795, 3.66666667],
       [4.92307692, 5.74358974, 5.33333333],
       [1.84615385, 2.15384615, 2.        ],
       [0.30769231, 0.35897436, 0.33333333]]))